In [ ]:
#Open API Fine Tuning Dashboard URL: https://platform.openai.com/finetune/

In [3]:
from dotenv import load_dotenv
from openai import OpenAI
import os

# --- Load API Key ---
load_dotenv(override=True, dotenv_path="../.env")
my_api_key = os.getenv("OPENAI_API_KEY")


client = OpenAI(api_key=my_api_key)

In [4]:
with open("data/training_data.jsonl", "rb") as f:
    training_file_obj = client.files.create(file=f, purpose="fine-tune")

print("Uploaded training file ID:", training_file_obj.id)

with open("data/validation_data.jsonl", "rb") as f:
    validation_file_obj = client.files.create(file=f, purpose="fine-tune")

print("Uploaded validation file ID:", validation_file_obj.id)

Uploaded training file ID: file-2W771GWpB8DpfMJWc3XxGA
Uploaded validation file ID: file-NpiidGq1KuVJjSAU1qQ39R


In [5]:
# Create a fine-tuning job
job = client.fine_tuning.jobs.create(
    training_file=training_file_obj.id,       # The file ID you uploaded
    validation_file=validation_file_obj.id,     # Optional
    model="gpt-4.1-nano-2025-04-14",               # or "gpt-3.5-turbo", etc.
    suffix="brand-customer-support"       # Optional model name suffix
)

print("Fine-tune job created:", job.id)

Fine-tune job created: ftjob-mx7v1f6BzhRKLoroPblGbfaO


In [10]:
jobs = client.fine_tuning.jobs.list(limit=5)
for j in jobs.data:
    print(j.id, j.status, j.fine_tuned_model)

ftjob-mx7v1f6BzhRKLoroPblGbfaO running None


In [13]:
def ask_question_without_finetuning(prompt):
    print(f"User asked: {prompt}")
    response = client.chat.completions.create(
        model="gpt-5-nano",
        messages=[
            {"role": "system", "content": "You are a helpful assistant. Answer as concisely as possible."},
            {"role": "user", "content": prompt}
        ]
    )
    print (response)
    return response.choices[0].message.content  

def ask_question_with_finetuning(prompt):
    print(f"User asked: {prompt}")
    response = client.chat.completions.create(
        model="ft:gpt-4.1-nano-2025-04-14:personal:brand-customer-support:CjqwilM3",
        messages=[
            {"role": "system", "content": "You are a helpful assistant. Answer as concisely as possible."},
            {"role": "user", "content": prompt}
        ]
    )
    print (response)
    return response.choices[0].message.content  

In [ ]:
import time
while True:
    # Ask user for a question
    user_prompt = input("Ask something: ")

    if (user_prompt.lower() != 'quit'):
        # Get and print the response
        response = ask_question_without_finetuning(user_prompt)
        print("\n[WITHOUT FINE TUNING] OpenAI says:", response)

        response = ask_question_with_finetuning(user_prompt)
        print("\n[WITH FINE TUNING] OpenAI says:", response)

        # add delay of 3 seconds
        time.sleep(3)
    else:
        break   

User asked: I have not recieved my refund
ChatCompletion(id='chatcmpl-CjrAHoh7Q0WUVzWEw5GSVDwOjj0as', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content="Sorry you're having trouble with your refund. I can help you troubleshoot. A few quick questions to tailor the steps:\n\n- Which company or platform is this (e.g., retailer, airline, app, marketplace)?\n- Do you have an order number or refund reference?\n- When was the refund promised or issued, and how long has it been?\n- How did you pay (credit/debit card, PayPal, store credit, etc.)?\n\nMeanwhile, try these steps:\n- Check your account: look in the order’s history for a refunds/credit item or status update.\n- Check your email or messages for a refund confirmation.\n- Verify the original payment method: refunds are usually sent back to the original method.\n- Note typical timelines (may vary by platform):\n  - Card refunds: several business days after the refund is processed.\n  - P